In [ ]:
# Copyright (c) 2024, ETH Zurich

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import os
import scipy.stats as stats
import scipy.ndimage
from tqdm import tqdm
import spekpy as spk
import h5py

from scipy import interpolate

In [ ]:
rave_sim_dir = Path('path/to/rave-sim')
simulations_dir = Path('path/to/data/output')
scratch_dir = simulations_dir

sys.path.insert(0, str(rave_sim_dir / "nist_lookup"))
from nist_lookup.xraydb_plugin import xray_delta_beta

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util
import propagation


In [ ]:
def calculate_G1_height(eng):
    # constants
    h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
    c_0 = 299792458 # speed of light in m / s
    eV_to_joule = 1.602176634*10**(-19)
    N_A = 6.02214086 * 10**23 #[1/mol]
    
    lambda_ = h * c_0 / (eng*eV_to_joule)
    delta_diff = xray_delta_beta('Au', 19.32, eng)[0] -  xray_delta_beta('Si', 2.34, eng)[0]
    height = np.pi  * lambda_ / (2*np.pi * delta_diff)

    return height


def signal_retrieval_least_squares(data, period=None, axis=-1):
    if axis != -1:
        data = np.moveaxis(data, axis, -1)

    nsteps = data.shape[-1]

    if period is None:
        period = nsteps

    phi = np.linspace(0, 2 * np.pi * nsteps / (period), nsteps, endpoint=False)
    M = np.c_[np.sin(phi), np.cos(phi), np.ones(nsteps)]
    res, chi2, _rank, _sing_vals = np.linalg.lstsq(M, data.reshape((-1, nsteps)).T, rcond=-1)

    res = res.T.reshape((*data.shape[:-1], -1))

    dabs = res[...,2]
    dphase = -np.arctan2(res[...,0], res[...,1])
    dvis = np.sqrt(res[...,0]**2 + res[...,1]**2) / dabs

    # normalization to the total number of counts
    dabs *= nsteps

    return dabs, dphase, dvis, np.nanmean(chi2)


def calculate_pixel_intensity(x, fringe, pxEdges, statistics = 'sum'):
    fringeStats = stats.binned_statistic(x, fringe, bins=pxEdges, statistic = statistics)
    return fringeStats.statistic

def perform_binned_signal_retrieval(x, wf, pxSize, nrSteps, plot_curve = True):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = []
    for i in range(nrSteps):
        int_px.append(calculate_pixel_intensity(x, wf[i,:], pxEdges))
    int_px = np.asarray(int_px)

    trans, phase, vis, _ = signal_retrieval_least_squares(int_px, period = nrSteps, axis = 0)
    if plot_curve:
        plot_curves(trans, phase, vis, pxEdges)
    return int_px, trans, phase, vis, pxEdges

def perform_binning(x, wf, pxSize):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = calculate_pixel_intensity(x, wf, pxEdges, statistics = 'mean')
    return int_px

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def get_subdict(dict_, idx):
    sub_dict = {}
    for key in dict_.keys():
        sub_dict[key] = dict_[key][idx]
    return sub_dict

def calc_Vis_theoretical(eng, Edes,m):
    V = 2/np.pi * np.abs(np.sin(np.pi / 2 * Edes / eng)**2 * np.sin(m * np.pi / 2 * Edes / eng))
    return V

In [ ]:
# constants
h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
c_0 = 299792458 # speed of light in m / s
eV_to_joule = 1.602176634*10**(-19)
N_A = 6.02214086 * 10**23 #[1/mol]
E_des = 46000

lambda_ = h * c_0 / (E_des*eV_to_joule)
p2 = 4.2*10**(-6)
p0 = p1 = p2

Dn_5 = 5*p2**2/(2*lambda_) / 2

z_g0 = 0.1
z_g1 = z_g0 + Dn_5
z_g2 = z_g0 + 2*Dn_5
z_detector = z_g2 + 0.01

h0 = h2 = 180e-6
h1 = 59e-6

print("Z0: ", z_g0)
print("Z1: ", z_g1)
print("Z2: ", z_g2)
print("Z Detector: ", z_detector)

print(Dn_5)

In [ ]:
N = 2**26
max_energy = 70000
dx = propagation.max_dx(z_g0, 200e-6, N, propagation.convert_energy_wavelength(max_energy))

In [ ]:
dx**2 / lambda_ * (1 / 0.1)

In [ ]:
s = spk.Spek(kvp=70, dk = 0.1, th = 10) # Create a spectrum
s.multi_filter((('Be', 0.15), ('Al', 3))) # Create a spectrum
k, f = s.get_spectrum(edges=True) # Get the spectrum

energyRange = [4000, 70000]
dE = 100
filtering = 0.000

energies = np.arange(5, 70+0.1, 0.1)*1e3


tube_spectrum_txt = interpolate.interp1d(k*1e3, f, fill_value = 'extrapolate')
spec_txt = tube_spectrum_txt(energies)

with h5py.File('spectrum_70_spekpy_filtered_3mmAl.h5', 'w') as h5:
    h5.create_dataset('pdf', data =  spec_txt/ np.sum(spec_txt))
    h5.create_dataset('energy', data = energies) 

In [ ]:
plt.plot(energies, spec_txt)

In [ ]:
structure_shift = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

grid_path_g0 = ['path_to_shift_phantom/Phase_shift_42_4.npy']
source_sizes = np.array([5, 15, 50, 200])*1e-6

dx_g0 = 1.0151416062788299e-08
dz_g0 = 180e-6 / 20

In [ ]:
len(grid_path_g0) * len(source_sizes)

In [ ]:
all_sims = []
for grid_path in grid_path_g0:
    for ss in source_sizes:

        config_dict = {
                    "sim_params": {
                        "N": N,
                        "dx": dx,
                        "z_detector": z_g2 + 500e-6,
                        "detector_size": 0.004,
                        "detector_pixel_size_x": 1e-7,
                        "detector_pixel_size_y": 1.0,
                        "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
                    },
                    "use_disk_vector": False,
                    "save_final_u_vectors": False,
                    "dtype": "c8",
                    "multisource": {
                        "type": "points",
                        "energy_range": [11000, 70000],
                        "x_range": [-ss.item(), ss.item()],
                        "z": 0.0,
                        "nr_source_points": 200,
                        "seed": 1,
                        "spectrum": 'path/to/simulation_spectrum_file.h5',
                    },
                    "elements": [
                        {
                            "type": "sample",
                            "z_start": z_g0,
                            "pixel_size_x": dx_g0,
                            "pixel_size_z": dz_g0,
                            "grid_path": grid_path,
                            "materials": [["Au", 19.32],["C5H8O2", 1.19]],
                            "x_positions": [0.0],
                        },
                        {
                            "type": "grating",
                            "pitch": 4.2*1e-6,
                            "dc": [1.0, 1.0],
                            "z_start": z_g0 + 180*1e-6,
                            "thickness": 500*1e-6 - h0,
                            "nr_steps": 1,
                            "x_positions": [0.0],
                            "substrate_thickness": 0.0,
                            "mat_a": ["C", 2.26],
                            "mat_b": None,
                            "mat_substrate": None
                        },           
                        {
                            "type": "grating",
                            "pitch": p1,
                            "dc": [0.5, 0.5],
                            "z_start": z_g1,
                            "thickness": h1,
                            "nr_steps": 10,
                            "x_positions": [0.0],
                            "substrate_thickness": 200 * 1e-6 - h1,
                            "mat_a": ["Si", 2.34],
                            "mat_b": None,
                            "mat_substrate": ["Si", 2.34],
                        },
                        {
                            "type": "grating",
                            "pitch": p2,
                            "dc": [0.5, 0.5],
                            "z_start": z_g2,
                            "thickness": h2,
                            "nr_steps": 30,
                            "x_positions": (np.arange(5) * p2/5).tolist(),
                            "substrate_thickness": 500*1e-6 - h2,
                            "mat_a": ["C5H8O2", 1.19],
                            "mat_b": ["Au", 19.32],
                            "mat_substrate": ["C", 2.26],
                        },
                    ],
            }
        sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)

        for i in tqdm(range(config_dict["multisource"]["nr_source_points"])):
            os.system(f"CUDA_VISIBLE_DEVICES=0 path/to/rave-sim/fast-wave/build-Release/fastwave -s {i} {sim_path}")

        all_sims.append(sim_path)

In [ ]:
x_units = np.linspace(-25 * 75, 25 * 75, 50)

In [ ]:
vis_all = []
for s_path in all_simulations:
    sim_path = s_path
    pxSize = 75e-6
    config_dict = config.load(Path(str(sim_path) + '/config.yaml'))
    sp = config_dict["sim_params"]
    detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
    pixel_rectangle = np.abs(detector_x) <= pxSize
    
    wavefronts_ref = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[11000, 70000])
    wf_ref = np.zeros_like(wavefronts_ref[0][0])
    
    energies = []
    for wave in wavefronts_ref:
        energies.append(wave[2])
        wf_ref += wave[0] #* np.exp(-mu_h2o(wave[2])*0.2)
    
    phase_steps = 5
    del wavefronts_ref
    
    convolved_ref = []
    for i in range(phase_steps):
        convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))
    
    convolved_ref = np.asarray(convolved_ref)
    
    int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
    int_px, trans_orig, phase_orig, vis_orig, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)

    vis_all.append(vis_ref)


In [ ]:
# matplotlib style
plt.style.use("default")

# set FIGWIDTH to latex's \textwidth
FIGWIDTH = 3
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 6
plt.rcParams["figure.figsize"] = (FIGWIDTH, FIGWIDTH * 2 / 3)
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.constrained_layout.use"] = "True"

# images
plt.rcParams["image.interpolation"] = "bicubic"
plt.rcParams["image.cmap"] = "Greys_r"

# axes
# plt.rcParams["axes.spines.right"] = False
# plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.edgecolor"] = "0.7"
plt.rcParams["axes.linewidth"] = "1"

# legend
plt.rcParams["legend.frameon"] = True

plt.rcParams["lines.markersize"] = 3
# plt.rcParams["lines.markerfacecolor"] = "white"
# Okabe-Ito palette
plt.rcParams["axes.prop_cycle"] = plt.cycler(
    color=[
        "#000000",
        "#E69F00",
        "#56B4E9",
        "#009E73",
        "#F0E442",
        "#0072B2",
        "#D55E00",
        "#CC79A7",
    ],
    #marker=["o", "^", "s", "p", "D", "v", "v", "d"],
)

plt.rcParams['axes.grid'] = False

In [ ]:
x_pixel = np.linspace(-vis_all[0].shape[0]/2 * 75, vis_all[0].shape[0]/2*75, vis_all[0].shape[0])

In [ ]:
source_sizes = np.array([5, 15, 50, 200])*1e-6

In [ ]:
vis_small_crack = np.load('dvis_small_crack_all.npy')
vis_medium_crack = np.load('dvis_medium_crack_all.npy')
vis_large_crack = np.load('dvis_large_crack_all.npy')

x_units = np.linspace(-25 * 75, 25 * 75, 50)

In [ ]:
plt.figure(figsize=(3,2.5))
for idx in range(len(source_sizes)):
    if idx == 2:
        continue
    
    plt.plot(x_pixel/1e3, vis_all[idx]*0.91, label = f"{2*source_sizes[idx]*1e6:.2f}µm S")

plt.plot(x_pixel[2:-1]/1e3, np.mean(vis_small_crack[70:, :], axis = 0), marker = 'o', label = '10µm M', linewidth = 1.0, linestyle = ':', color = "#000000", markeredgecolor = 'black', markeredgewidth = 0.1)
plt.plot(x_pixel[2:-1]/1e3, np.mean(vis_medium_crack[70:, :],axis = 0), marker = '^', label = '30µm M', linewidth = 1.0, linestyle = ':', color = "#009E73", markeredgecolor = 'black',  markeredgewidth = 0.2)
plt.plot(x_pixel[2:-1]/1e3, np.mean(vis_large_crack[70:90, :], axis = 0), marker = '^', label = '400µm M', linewidth = 1.0, linestyle = ':', color = "#0072B2", markeredgecolor = 'black',  markeredgewidth = 0.2)


plt.title('Visibility profile')
plt.ylabel('Visibility', fontsize = 6)
plt.xlabel('Distance on detector [mm]', fontsize = 6)
plt.xlim(-1, 1)
plt.ylim(0.033, 0.18)
legend = plt.legend(loc = 'best', fontsize = 5, bbox_to_anchor=(0.5, 0.02, 0.5, 0.5))
legend.get_frame().set_alpha(0.5)
